![download.png](download.png)


## Import Packages

In [1]:
#Import Python packages
import pandas as pd
import plotly.express as px
import json
import sys
import cachetools

# Import Snowflake modules
from snowflake.snowpark import Session
import snowflake.snowpark.functions as F
import snowflake.snowpark.types as T
from snowflake.snowpark import Window

## Connect to Snowflake

In [22]:
# Get account credentials from a json file

'''with open("data_scientist_auth.json") as f:
    data = json.load(f)
    username = data["username"]
    password = data["password"]
    account = data["account"]'''

# Specify connection parameters
connection_parameters = {
    "account": 'LGZKWLH-EB17683',
    "user": 'NUGGETIERER',
    "password": 'gOldMineR420',
    "role": "tasty_data_scientist",
    "warehouse": "tasty_ds_wh",
    "database": "frostbyte_tasty_bytes",
    "schema": "analytics",
}

# Create Snowpark session
session = Session.builder.configs(connection_parameters).create()

## Gathering Data as Snowpark DataFrame

In [23]:
#Country
tasty_bytes_country = session.table("frostbyte_tasty_bytes.raw_pos.country")
#Franchise
tasty_bytes_franchise = session.table("frostbyte_tasty_bytes.raw_pos.franchise")
#Location
tasty_bytes_location = session.table("frostbyte_tasty_bytes.raw_pos.location")
#Menu
tasty_bytes_menu = session.table("frostbyte_tasty_bytes.raw_pos.menu")
#Order Detail
tasty_bytes_order_detail = session.table("frostbyte_tasty_bytes.raw_pos.order_detail")
#Order Header
tasty_bytes_order_header = session.table("frostbyte_tasty_bytes.raw_pos.order_header")
#Truck
tasty_bytes_truck = session.table("frostbyte_tasty_bytes.raw_pos.truck")
#Customer Loyalty
tasty_bytes_customer_loyalty = session.table("frostbyte_tasty_bytes.raw_customer.customer_loyalty")


## Preview The Data

In [3]:
print("Number of rows:", tasty_bytes_country.count())
tasty_bytes_country.show()

NameError: name 'tasty_bytes_country' is not defined

In [6]:
print("Number of rows:", tasty_bytes_franchise.count())
tasty_bytes_franchise.show()

Number of rows: 335
--------------------------------------------------------------------------------------------------------------------------
|"FRANCHISE_ID"  |"FIRST_NAME"  |"LAST_NAME"  |"CITY"     |"COUNTRY"      |"E_MAIL"                     |"PHONE_NUMBER"  |
--------------------------------------------------------------------------------------------------------------------------
|1               |Corporate     |Owned        |San Mateo  |United States  |TastyBytes@Gmail.com         |434-928-2233    |
|2               |Annette       |Gates        |San Mateo  |United States  |Annette.Gates@yahoo.com      |949-148-5165    |
|3               |Steven        |Phelps       |San Mateo  |United States  |Steven.Phelps@aol.com        |746-288-2047    |
|4               |Sandra        |Dunn         |San Mateo  |United States  |Sandra.Dunn@ymail.com        |043-798-0237    |
|5               |Diane         |Cruz         |San Mateo  |United States  |Diane.Cruz@gmail.com         |850-879-6541  

In [7]:
print("Number of rows:", tasty_bytes_location.count())
tasty_bytes_location.show()

Number of rows: 13093
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"LOCATION_ID"  |"PLACEKEY"           |"LOCATION"                                          |"CITY"      |"REGION"                               |"ISO_COUNTRY_CODE"  |"COUNTRY"  |
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|1001           |222-222@4hf-nch-k2k  |Seasoners Fine Foods                                |Manchester  |Greater Manchester                     |GB                  |England    |
|1002           |222-222@4hf-ndn-4sq  |HM Music Promotions                                 |Manchester  |Greater Manchester                     |GB                  |England    |
|1003           |222-222@4hf-ndp-grk  |G J Hull                                    

In [8]:
print("Number of rows:", tasty_bytes_menu.count())
tasty_bytes_menu.show()

Number of rows: 100
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"MENU_ID"  |"MENU_TYPE_ID"  |"MENU_TYPE"  |"TRUCK_BRAND_NAME"  |"MENU_ITEM_ID"  |"MENU_ITEM_NAME"    |"ITEM_CATEGORY"  |"ITEM_SUBCATEGORY"  |"COST_OF_GOODS_USD"  |"SALE_PRICE_USD"  |"MENU_ITEM_HEALTH_METRICS_OBJ"     |
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|10001      |1               |Ice Cream    |Freezing Point      |10              |Lemonade            |Beverage         |Cold Option         |0.6500               |3.5000            |{                                  |
|           |                |             |                    |                |                  

In [9]:
print("Number of rows:", tasty_bytes_order_detail.count())
tasty_bytes_order_detail.show()

Number of rows: 673655465
--------------------------------------------------------------------------------------------------------------------------------------------------------
|"ORDER_DETAIL_ID"  |"ORDER_ID"  |"MENU_ITEM_ID"  |"DISCOUNT_ID"  |"LINE_NUMBER"  |"QUANTITY"  |"UNIT_PRICE"  |"PRICE"  |"ORDER_ITEM_DISCOUNT_AMOUNT"  |
--------------------------------------------------------------------------------------------------------------------------------------------------------
|305920870          |112871854   |141             |NULL           |2              |2           |17.0000       |34.0000  |NULL                          |
|305920871          |112871854   |142             |NULL           |3              |1           |18.0000       |18.0000  |NULL                          |
|305920872          |112871855   |141             |NULL           |0              |1           |17.0000       |17.0000  |NULL                          |
|305920873          |112871855   |142             |NULL 

In [10]:
print("Number of rows:", tasty_bytes_order_header.count())
tasty_bytes_order_header.show()

Number of rows: 248201269
-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"ORDER_ID"  |"TRUCK_ID"  |"LOCATION_ID"  |"CUSTOMER_ID"  |"DISCOUNT_ID"  |"SHIFT_ID"  |"SHIFT_START_TIME"  |"SHIFT_END_TIME"  |"ORDER_CHANNEL"  |"ORDER_TS"           |"SERVED_TS"  |"ORDER_CURRENCY"  |"ORDER_AMOUNT"  |"ORDER_TAX_AMOUNT"  |"ORDER_DISCOUNT_AMOUNT"  |"ORDER_TOTAL"  |
-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|376615852   |353         |8155.0         |NULL           |NULL           |200389070   |09:00:00            |15:00:00          |

In [11]:
print("Number of rows:", tasty_bytes_truck.count())
tasty_bytes_truck.show()

Number of rows: 450
-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"TRUCK_ID"  |"MENU_TYPE_ID"  |"PRIMARY_CITY"  |"REGION"    |"ISO_REGION"  |"COUNTRY"      |"ISO_COUNTRY_CODE"  |"FRANCHISE_FLAG"  |"YEAR"  |"MAKE"        |"MODEL"           |"EV_FLAG"  |"FRANCHISE_ID"  |"TRUCK_OPENING_DATE"  |
-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|1           |1               |San Mateo       |California  |CA            |United States  |US                  |0                 |2009    |Ford_         |Step Van          |0          |1               |2019-01-01            |
|2           |2               |San Mateo       |California  |CA     

In [12]:
print("Number of rows:", tasty_bytes_customer_loyalty.count())
tasty_bytes_customer_loyalty.show()

Number of rows: 222540
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"CUSTOMER_ID"  |"FIRST_NAME"  |"LAST_NAME"  |"CITY"          |"COUNTRY"  |"POSTAL_CODE"  |"PREFERRED_LANGUAGE"  |"GENDER"     |"FAVOURITE_BRAND"  |"MARITAL_STATUS"  |"CHILDREN_COUNT"  |"SIGN_UP_DATE"  |"BIRTHDAY_DATE"  |"E_MAIL"                  |"PHONE_NUMBER"  |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|1              |Rodolfo       |Tucker       |Hamburg         |Germany    |21129          |German                |Female       |NULL               |Single            |0           

## Data Exploration

In [6]:
order_header_select = tasty_bytes_order_header

In [7]:
order_header_select.show()

-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"ORDER_ID"  |"TRUCK_ID"  |"LOCATION_ID"  |"CUSTOMER_ID"  |"DISCOUNT_ID"  |"SHIFT_ID"  |"SHIFT_START_TIME"  |"SHIFT_END_TIME"  |"ORDER_CHANNEL"  |"ORDER_TS"           |"SERVED_TS"  |"ORDER_CURRENCY"  |"ORDER_AMOUNT"  |"ORDER_TAX_AMOUNT"  |"ORDER_DISCOUNT_AMOUNT"  |"ORDER_TOTAL"  |
-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|340686001   |294         |7891.0         |NULL           |NULL           |200325777   |15:00:00            |22:00:00          |NULL             |2022-06-

In [8]:
customer_count = order_header_select[order_header_select["ORDER_TS"] > "2022-9-14"]
customer_after = order_header_select[order_header_select["ORDER_TS"] > "2022-10-24"]

In [9]:
cust_bef = customer_count.to_pandas()

In [10]:
cust_aft = customer_after.to_pandas()

In [11]:
print(cust_bef["CUSTOMER_ID"].nunique())
print(cust_aft["CUSTOMER_ID"].nunique())

218735
120274


In [27]:
churn_percent = ((218735 - 120274) / 218735) * 100
print(churn_percent)

45.01382951973849


In [16]:
tasty_bytes_order_header.select(F.countDistinct("ORDER_ID")).show()

--------------------------------------
|"COUNT( DISTINCT ""CUSTOMER_ID"")"  |
--------------------------------------
|222540                              |
--------------------------------------



In [18]:
tasty_bytes_order_detail.select(F.countDistinct("ORDER_DETAIL_ID")).show()

------------------------------------------
|"COUNT( DISTINCT ""ORDER_DETAIL_ID"")"  |
------------------------------------------
|673655465                               |
------------------------------------------



In [26]:
tasty_bytes_order_detail.select(F.countDistinct("DISCOUNT_ID")).show()

--------------------------------------
|"COUNT( DISTINCT ""DISCOUNT_ID"")"  |
--------------------------------------
|0                                   |
--------------------------------------



In [27]:
tasty_bytes_order_header.select(F.countDistinct("DISCOUNT_ID")).show()

--------------------------------------
|"COUNT( DISTINCT ""DISCOUNT_ID"")"  |
--------------------------------------
|0                                   |
--------------------------------------



In [21]:
session.close()